In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
    ]

enc = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

print("model device:", next(model.parameters()).device)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9
# )

outputs = model.generate(
    input_ids=enc["input_ids"],
    attention_mask=enc.get("attention_mask", None),
    max_new_tokens=512,          # 처음엔 2048 말고 256~512 추천
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)


prompt_len = enc["input_ids"].shape[-1]
print(tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True))
# print(tokenizer.decode(outputs[0]))
# print(tokenizer.decode(outputs[0][input_ids.shape[-1]:]))
# print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


model device: cuda:0
철수가 20개의 연필을 가지고 있었고 영희가 절반을 가져가면, 영희가 가져간 연필의 갯수는 20 / 2 = 10개입니다.

철수가 남은 연필의 갯수는 20 - 10 = 10개입니다.

민수가 남은 5개의 연필을 가져가면, 철수가 남은 연필의 갯수는 10 - 5 = 5개입니다.

따라서 철수가 남은 연필의 갯수는 5개입니다.


In [47]:
def textAugmentation(model, PROMPT, FEWSHOT, instruction):
    import time
    import torch

    torch.cuda.synchronize()   # ✅ GPU 대기
    start_time = time.time()
    
    messages = [
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": FEWSHOT},
        {"role": "user", "content": instruction},
    ]
    
    
    enc = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    outputs = model.generate(
        input_ids=enc["input_ids"],
        attention_mask=enc.get("attention_mask", None),
        max_new_tokens=512,          # 처음엔 2048 말고 256~512 추천
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    prompt_len = enc["input_ids"].shape[-1]

    result = tokenizer.decode(outputs[0][prompt_len:], skip_special_tokens=True)

    torch.cuda.synchronize()   # ✅ 다시 대기
    end_time = time.time()
    
    elapsed = end_time - start_time
    print(f"⏱ GPU 포함 실행 시간: {elapsed:.3f}초")
    
    return result

In [135]:
PROMPT = """
너는 두 사람이 일상적으로 대화하는 한국어 문장을 만들어줘야해.
두 사람이 대화하는 문장을 만들어줘.
화자 이름은 쓰지 마.
자연스럽고 상황에 맞는 말투(반말, 존댓말)로 대화해.
안내나 설명 없이 답변만 출력해
"""

FEWSHOT = """
답변예시1:
이번 주말에 뭐 할까?
영화 한 편 보는 건 어때?
좋다, 팝콘도 사 가자.
그럼 내가 예매할게.

답변예시2:
오늘 회사에서 너무 피곤했어.
무슨 일 있었어?
회의가 계속 있어서 힘들었어.
오늘은 일찍 쉬어야겠다.

답변예시3:
요즘 어디 여행 가고 싶어?
바다 있는 곳이 좋을 것 같아.
부산이나 제주도 괜찮겠다.
가까운 곳으로 가자.

답변예시4:
요즘 운동 좀 하는 것 같더라.
응, 살 좀 빼려고.
어떤 운동해?
퇴근하고 헬스장 가.

답변예시5:
오늘 날씨 진짜 좋다.
그러게, 산책하기 딱이야.
끝나고 좀 걸을까?
좋아, 커피도 마시자.
"""

topic = "카드 분실 신고"

instruction = f"""


{topic}를/을 주제로 6줄의 대화 1개만 만들어줘.
"""

In [136]:
print(textAugmentation(model, PROMPT, FEWSHOT, instruction))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 1.786초
답변예시:
카드를 잃어버렸어.
무슨 일이 있었어?
아마도 집에 남아있었어.
그럼 그곳에서 찾아보자.
저도 도와줄게.


In [142]:
import random

TOPICS = [
    # 음식·식사
    "점심 메뉴 못 정해서 고민",
    "혼밥 vs 같이 먹기",
    "배달앱 메뉴 추천 요청",
    "비 오는 날 음식 고르기",
    "다이어트 중 메뉴 고민",
    "야식 먹을지 말지 고민",
    "회사 근처 맛집 이야기",
    "집에 재료 없을 때 요리 고민",
    "카페 메뉴 고르기",
    "신메뉴 후기 공유",

    # 컨디션·생활
    "야근 후 너무 피곤함",
    "잠 설쳐서 졸림",
    "감기 기운 있음",
    "운동 시작했는데 힘듦",
    "허리나 목 통증",
    "스트레스 받아서 지침",
    "주말에 몰아서 잠 자기",
    "불면증 고민",
    "체력 떨어진 느낌",
    "건강검진 결과 이야기",

    # 약속·이동
    "약속 시간 변경 요청",
    "지각해서 사과 연락",
    "길 막혀서 늦음",
    "만날 장소 헷갈림",
    "어디쯤 왔는지 확인",
    "약속 취소 상황",
    "비 와서 약속 고민",
    "막차 걱정",
    "주차 자리 못 찾음",
    "귀가 중 통화",

    # 감정·관계
    "상사 때문에 스트레스",
    "친구와 싸운 후 고민",
    "연인과 오해 발생",
    "시험이나 면접 불안",
    "실패해서 우울함",
    "잘한 일 자랑",
    "위로 요청",
    "자신감 부족",
    "진로 고민",
    "번아웃 상태",

    # 소비·서비스
    "음식 배달 지연 항의",
    "상품 불량 교환 문의",
    "환불 규정 질문",
    "예약 변경 요청",
    "택배 분실 문의",
    "쿠폰 사용법 질문",
    "포인트 적립 문의",
    "영수증 재발급 요청",
    "가격 비교 상담",
    "AS 접수 문의",

    # 전화·메신저
    "부재중 전화 해명",
    "배터리 없어서 급통화",
    "통화 소리 끊김 문제",
    "끊긴 전화 재연결",
    "늦은 밤 전화 사과",
    "급하게 위치 물어봄",
    "운전 중 통화",
    "잘못 걸린 전화",
    "문자 vs 통화 선택",
    "통화 소리가 작은 문제",

    # 긴급·문제
    "지갑 분실",
    "휴대폰 분실",
    "교통사고 목격",
    "차 고장",
    "집 열쇠 분실",
    "길 잃음",
    "카드 분실 신고",
    "사기 의심 상황",
    "소음 신고",
    "위급 상황 신고",

    # 취미 생활
    "요즘 취미생활 뭐 하는지 이야기",
    "새로운 취미 시작 고민",
    "운동을 취미로 시작한 이야기",
    "게임을 취미로 즐기는 이야기",
    "드라마나 영화 보는 취미",
    "음악 듣기나 연주 취미 이야기",
    "책 읽는 취미 이야기",
    "사진이나 영상 찍는 취미",
    "요리나 베이킹 취미 이야기",
    "혼자 즐기는 취미생활 이야기"
]


for _ in range(10):
    topic = random.choice(TOPICS)

    instruction = f"""{topic}를/을 주제로 6줄의 대화 1개만 만들어줘."""
    
    print(textAugmentation(model, PROMPT, FEWSHOT, instruction))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 1.931초
아, 잠이 좀 쉴 수 있어.
저도 많이 졸리고 있어.
저녁에 잠들면 안 돼.
어제까지도 잘 자지.
내일도 잘 자야겠다.
그럼 오늘도 잘 수 있을 거야.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 1.968초
답변예시:
집에 재료 없을 때 요리 고민을 하네.
어떤 식을 할까?
파는 것만 있니, 파전이나 파스타가 좋을 것 같아.
파전做는 재료가 많아.
파전으로 가자.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

⏱ GPU 포함 실행 시간: 1.704초
답변예시:
그게 잘한 일이야.
회사에서 정말 큰 성과를 내웠어.
제가 만든 프로젝트가 큰 인정을 받았어.
팀원들과 함께 노력했어.
그로 인해 저의 경력도 크게 발전했어.
이제 더 많은 기회가 열릴 것 같아.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 1.933초
답변예시:
건강검진 결과 나왔어.
어떤 결과였어?
혈압 높아서 약을 먹어야겠어.
네, 잘 해. 약을 먹고 잘라야겠어.
약을 먹고 다음 검진도 잘 하자.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


⏱ GPU 포함 실행 시간: 2.850초
스트레스 받아서 지침을 주제로 6줄의 대화
아, 스트레스 받아서 어떻게 해?
저도 스트레스가 많이 받고 있어.
그럼 어떤 방법이 좋을까?
요즘에는 meditation이나 yoga가 좋다고 들었어.
그럼 오늘은 meditation해볼게.
그럼 오늘은 스트레스가 많이 줄어들겠어.
⏱ GPU 포함 실행 시간: 2.235초
답변예시:
회사 근처에 맛집이 있어.
어떤 맛집이 좋아?
오늘은 간장집으로 가자.
그럼 우리今天 가자.
가면 한 달에 한 번씩 가야겠어.
그럼 오늘은 간장집으로 가자.


NameError: name 'random' is not defined